In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
import argparse
import yaml
import os
import os.path as osp
from utils import get_logger
from model import build_model
from datasets import build_data_loader
from criterions import build_criterion
from trainer import build_trainer, build_eval
from utils import *
import numpy as np
from tqdm import tqdm


In [2]:
# Simulate the CLI arguments
args_list = [
    '--config', './configs/miniroad_ced.yaml',
    '--no_flow',
]

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
class CEDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        data = self.data[index]
        label = self.labels[index]
        return data, label

    def __len__(self):
        return len(self.data)
    

In [5]:
n_sensor_dataset = 4000
batch_size = 256
n_epochs = 5000
""" Load datasets """


train_data_file = './datasets/new_CE_dataset/new_ce5min_train_data_{}.npy'.format(n_sensor_dataset)
train_label_file = './datasets/new_CE_dataset/new_ce5min_train_labels_{}.npy'.format(n_sensor_dataset)
val_data_file = './datasets/new_CE_dataset/new_ce5min_val_data.npy'
val_label_file = './datasets/new_CE_dataset/new_ce5min_val_labels.npy'
test_data_file = './datasets/new_CE_dataset/new_ce5min_test_data.npy'
test_label_file = './datasets/new_CE_dataset/new_ce5min_test_labels.npy'

ce_train_data = np.load(train_data_file)
ce_train_labels = np.load(train_label_file)
ce_val_data = np.load(val_data_file)
ce_val_labels = np.load(val_label_file)
ce_test_data = np.load(test_data_file)
ce_test_labels = np.load(test_label_file)

print(train_data_file)
print(ce_train_data.shape, ce_train_labels.shape, ce_val_data.shape, ce_val_labels.shape, ce_test_data.shape, ce_test_labels.shape)

ce_train_dataset = CEDataset(ce_train_data, ce_train_labels)
ce_train_loader = DataLoader(ce_train_dataset, 
                             batch_size=batch_size, 
                             shuffle=True, 
                             )
ce_val_dataset = CEDataset(ce_val_data, ce_val_labels)
ce_val_loader = DataLoader(ce_val_dataset,
                            batch_size=batch_size,
                            shuffle=False, 
                            )
ce_test_dataset = CEDataset(ce_test_data, ce_test_labels)
ce_test_loader = DataLoader(ce_test_dataset, 
                            batch_size=batch_size, 
                            shuffle=False
                            )


./datasets/new_CE_dataset/new_ce5min_train_data_4000.npy
(4000, 60, 128) (4000, 60) (2000, 60, 128) (2000, 60) (2000, 60, 128) (2000, 60)


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./configs/miniroad_ced.yaml')
parser.add_argument('--eval', type=str, default=None)
parser.add_argument('--amp', action='store_true')
parser.add_argument('--tensorboard', action='store_true')
parser.add_argument('--lr_scheduler', action='store_true')
parser.add_argument('--no_rgb', action='store_true')
parser.add_argument('--no_flow', action='store_true')
args = parser.parse_args(args_list)

# combine argparse and yaml
opt = yaml.load(open(args.config), Loader=yaml.FullLoader)
opt.update(vars(args))
cfg = opt

set_seed(20)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

identifier = f'{cfg["model"]}_{cfg["data_name"]}_{cfg["feature_pretrained"]}_flow{not cfg["no_flow"]}'
result_path = create_outdir(osp.join(cfg['output_path'], identifier))
logger = get_logger(result_path)
logger.info(cfg)

testloader = ce_test_loader
model = build_model(cfg, device)
evaluate = build_eval(cfg)
if args.eval != None:
    model.load_state_dict(torch.load(args.eval))
    mAP = evaluate(model, testloader, logger)
    logger.info(f'{cfg["task"]} result: {mAP*100:.2f} m{cfg["metric"]}')
    exit()
    


{'model': 'MiniROAD', 'data_name': 'THUMOS', 'task': 'OAD', 'loss': 'NONUNIFORM', 'metric': 'F1', 'optimizer': 'AdamW', 'device': 'cuda', 'feature_pretrained': 'ced', 'root_path': './data/THUMOS', 'rgb_type': 'ced', 'flow_type': 'ced', 'annotation_type': 'target_perframe', 'video_list_path': 'data_info/video_list.json', 'output_path': './output', 'window_size': 1, 'batch_size': 256, 'test_batch_size': 1, 'num_epoch': 5000, 'lr': 0.0001, 'weight_decay': 0.05, 'num_workers': 4, 'dropout': 0.2, 'num_classes': 11, 'embedding_dim': 256, 'hidden_dim': 256, 'num_layers': 2, 'stride': 4, 'config': './configs/miniroad_ced.yaml', 'eval': None, 'amp': False, 'tensorboard': False, 'lr_scheduler': False, 'no_rgb': False, 'no_flow': True}


In [7]:
summary(model)

Layer (type:depth-idx)                   Param #
MROAD                                    --
├─ReLU: 1-1                              --
├─GRU: 1-2                               789,504
├─Sequential: 1-3                        --
│    └─Linear: 2-1                       33,024
│    └─LayerNorm: 2-2                    512
│    └─ReLU: 2-3                         --
│    └─Dropout: 2-4                      --
├─Sequential: 1-4                        --
│    └─Linear: 2-5                       2,827
Total params: 825,867
Trainable params: 825,867
Non-trainable params: 0

In [ ]:
# Define dummy input with shape (batch_size=256, sequence_length=60, features=128)
dummy_input = torch.randn(256, 60, 128)

# Move to correct device if needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
dummy_input = dummy_input.to(device)

# Pass through model
with torch.no_grad():
    output = model(dummy_input,flow_input=None)

print(output)
# Print input and output shapes
print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {output['logits'].shape}")

# Optional: model summary
try:
    summary(model, input_data=dummy_input)
except Exception as e:
    print(f"torchinfo summary error: {e}")

{'logits': tensor([[[ 2.0467e-02,  5.3975e-02, -8.8562e-02,  ..., -2.6340e-03,
          -5.0334e-02,  4.0932e-02],
         [ 1.4082e-02,  4.7149e-02, -1.1884e-01,  ..., -2.3188e-02,
          -1.4554e-02,  4.5983e-02],
         [ 4.7160e-02,  4.0824e-02, -1.6129e-01,  ..., -2.5377e-02,
           3.9020e-02,  7.8019e-02],
         ...,
         [ 6.3298e-02, -1.4868e-04, -2.8398e-01,  ..., -3.6084e-02,
           1.6436e-02,  1.3961e-01],
         [ 5.5924e-02,  1.7860e-02, -2.5739e-01,  ..., -2.9932e-02,
           3.4827e-02,  1.2925e-01],
         [ 3.0662e-02,  2.5545e-02, -2.2836e-01,  ..., -5.6979e-02,
           5.5429e-02,  1.2967e-01]],

        [[ 5.6828e-03,  6.0916e-02, -1.0629e-01,  ..., -1.8092e-02,
          -1.7680e-02,  4.5523e-02],
         [-1.6023e-02,  9.1496e-02, -1.3459e-01,  ..., -4.0122e-02,
           4.0282e-02,  7.6913e-02],
         [-1.9921e-02,  6.8413e-02, -1.3984e-01,  ..., -2.5369e-02,
           8.1301e-02,  1.2127e-01],
         ...,
         [ 8.2

: 

In [ ]:
trainloader = ce_train_loader
criterion = build_criterion(cfg, device)
train_one_epoch = build_trainer(cfg)
optim = torch.optim.AdamW if cfg['optimizer'] == 'AdamW' else torch.optim.Adam
optimizer = optim([{'params': model.parameters(), 'initial_lr': cfg['lr']}],
                    lr=cfg['lr'], weight_decay=cfg["weight_decay"])

scheduler = build_lr_scheduler(cfg, optimizer, len(trainloader)) if args.lr_scheduler else None
writer = SummaryWriter(osp.join(result_path, 'runs')) if args.tensorboard else None
scaler = torch.cuda.amp.GradScaler() if args.amp else None
total_params = sum(p.numel() for p in model.parameters())

logger.info(f'Dataset: {cfg["data_name"]},  Model: {cfg["model"]}')    
logger.info(f'lr:{cfg["lr"]} | Weight Decay:{cfg["weight_decay"]} | Window Size:{cfg["window_size"]} | Batch Size:{cfg["batch_size"]}') 
logger.info(f'Total epoch:{cfg["num_epoch"]} | Total Params:{total_params/1e6:.1f} M | Optimizer: {cfg["optimizer"]}')
logger.info(f'Output Path:{result_path}')

best_f1, best_epoch = 0, 0
for epoch in range(1, cfg['num_epoch']+1):
    epoch_loss = train_one_epoch(trainloader, model, criterion, optimizer, scaler, epoch, writer, scheduler=scheduler)
    # trainloader.dataset._init_features()
    f1 = evaluate(model, testloader, logger)
    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch
        torch.save(model.state_dict(), osp.join(result_path, 'ckpts', 'best.pth'))
    logger.info(f'Epoch {epoch} F1: {f1:.2f} | Best F1: {best_f1*100:.2f} at epoch {best_epoch}, iter {epoch*cfg["batch_size"]*len(trainloader)} | train_loss: {epoch_loss/len(trainloader):.4f}, lr: {optimizer.param_groups[0]["lr"]:.7f}')
    
os.rename(osp.join(result_path, 'ckpts', 'best.pth'), osp.join(result_path, 'ckpts', f'best_{best_f1*100:.2f}.pth'))

Dataset: THUMOS,  Model: MiniROAD
lr:0.0001 | Weight Decay:0.05 | Window Size:1 | Batch Size:256
Total epoch:5000 | Total Params:0.8 M | Optimizer: AdamW
Output Path:./output/MiniROAD_THUMOS_ced_flowFalse
Epoch:1 Training: 100%|██████████| 16/16 [00:00<00:00, 67.55it/s, lr: 0.0001000]
Processed 2000 frames in 0.0 seconds (40330.2 FPS)
Epoch 1 F1: 0.00 | Best F1: 0.00 at epoch 0, iter 4096 | train_loss: 0.0195, lr: 0.0001000


(2000, 60) (2000, 60)
NUM FRAMES: 120000


Epoch:2 Training: 100%|██████████| 16/16 [00:00<00:00, 116.94it/s, lr: 0.0001000]
Processed 2000 frames in 0.0 seconds (62793.7 FPS)
Epoch 2 F1: 0.00 | Best F1: 0.00 at epoch 0, iter 8192 | train_loss: 0.0165, lr: 0.0001000


(2000, 60) (2000, 60)
NUM FRAMES: 120000


Epoch:3 Training: 100%|██████████| 16/16 [00:00<00:00, 129.36it/s, lr: 0.0001000]
Processed 2000 frames in 0.0 seconds (57487.7 FPS)
Epoch 3 F1: 0.03 | Best F1: 3.13 at epoch 3, iter 12288 | train_loss: 0.0150, lr: 0.0001000


...

Epoch:464 Training: 100%|██████████| 16/16 [00:00<00:00, 105.85it/s, lr: 0.0001000]
Processed 2000 frames in 0.1 seconds (33179.8 FPS)
Epoch 464 F1: 0.82 | Best F1: 84.16 at epoch 404, iter 1900544 | train_loss: 0.0000, lr: 0.0001000


(2000, 60) (2000, 60)
NUM FRAMES: 120000


Epoch:465 Training: 100%|██████████| 16/16 [00:00<00:00, 144.42it/s, lr: 0.0001000]
                                                  

(2000, 60) (2000, 60)
NUM FRAMES: 120000


Processed 2000 frames in 0.0 seconds (55090.4 FPS)
Epoch 465 F1: 0.82 | Best F1: 84.16 at epoch 404, iter 1904640 | train_loss: 0.0000, lr: 0.0001000
Epoch:466 Training: 100%|██████████| 16/16 [00:00<00:00, 112.50it/s, lr: 0.0001000]
Traceback (most recent call last):                
  File "/home/nesl/anaconda3/envs/miniroad/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 562, in multilabel_confusion_matrix
    y_pred = le.transform(y_pred)
RuntimeError: Object of type <class 'numpy.ndarray'> appears to be C subclassed NumPy array, void scalar, or allocated in a non-standard way.NumPy reserves the right to change the size of these structures. Projects are required to take this into account by either recompiling against a specific NumPy version or padding the struct and enforcing a maximum NumPy version.
Exception ignored in: 'PyDataMem_UserFREE'
Traceback (most recent call last):
  File "/home/nesl/anaconda3/envs/miniroad/lib/python3.10/site-packages/sklearn/met

import numpy as np
print(np.__version__)
print(np.float32)
